In [99]:
import os
import pandas as pd

dataset_file = 'D:\manualCDmanagement\codes\Projects\VMs\skl algorithms\\00_datasets\yerevan_hotels'
file_name = 'yerevan_hotels.csv'
df =pd.DataFrame(pd.read_csv(os.path.join(dataset_file, file_name)))

separate = ['Star Rating', 'Rating', 'Staff', 'Facilities', 'Location', 'Comfort', 'Cleanliness']

for column in df.columns:
    if column in separate:
        df[column] = df[column].fillna(df[column].mean())

# for column in df.columns:
#     if column not in separate and column != 'Price Per Day ($)':
#             df[column], _ = df[column].factorize()

for column in df.columns:
    if column not in separate and column not in {'Price Per Day ($)', 'Hotel Names'}:
            df[column] = df[column].map({'Yes':1, 'No':0})

In [100]:
df.head()

,Hotel Names,Star Rating,Rating,Free Parking,Fitness Centre,Spa and Wellness Centre,Airport Shuttle,Staff,Facilities,Location,Comfort,Cleanliness,Price Per Day ($)
0,Hayasa Hotel Yerevan,4.000000,9.0,0,1,0,1,9.2,9.0,9.1,9.5,9.4,88
1,Latar Hotel Yerevan,5.000000,8.1,0,1,1,1,8.4,8.3,7.4,8.7,8.5,87
2,Alex Hotel,4.000000,8.5,1,0,0,1,8.9,8.4,8.1,8.7,8.8,56
3,ENIGMA HOSTEL,3.891892,8.8,0,0,0,1,9.6,8.7,9.7,8.7,9.1,20
4,Ani Grand Hotel Yerevan,4.000000,8.8,0,1,0,0,8.9,8.7,9.4,9.1,9.0,130


In [101]:
df.isnull().sum()

Hotel Names                0
Star Rating                0
Rating                     0
Free Parking               0
Fitness Centre             0
Spa and Wellness Centre    0
Airport Shuttle            0
Staff                      0
Facilities                 0
Location                   0
Comfort                    0
Cleanliness                0
Price Per Day ($)          0
dtype: int64

In [102]:
from sklearn.model_selection import train_test_split
X = df.drop(['Hotel Names', 'Price Per Day ($)'], axis=1)
y = df['Price Per Day ($)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [103]:
# scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [157]:

from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(mse, rmse, r2)

coefficients = model.coef_
features = X.columns

coef_df = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})
coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()
coef_df_sorted = coef_df.sort_values(by='Abs_Coefficient', ascending=True)

# plt.figure(figsize=(10, 6))
# plt.barh(coef_df_sorted['Feature'], coef_df_sorted['Coefficient'])
# plt.xlabel('Coefficient Value')
# plt.ylabel('Feature Name')
# plt.title('Linear Regression Coefficients')
# plt.grid(True)

620.6701396982317 24.91325229066313 -0.45008022254273916


In [143]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
model_rf = RandomForestRegressor(n_estimators=35, random_state=42)

model_rf.fit(X_train_scaled, y_train)
y_pred_rf = model_rf.predict(X_test_scaled)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse)

print(mse_rf, rmse_rf, r2_rf)

524.0398185941044 24.91325229066313 -0.22432146830465705


In [160]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
import numpy as np
model_rf = RandomForestRegressor(n_estimators=35, random_state=42)

model_rf.fit(X_train_scaled, y_train)
y_pred_rf = model_rf.predict(X_test_scaled)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse)


# cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_mse = cross_val_score(model_rf, X_train_scaled, y_train, cv=kf, scoring='neg_mean_squared_error')
cross_val_r2 = cross_val_score(model_rf, X_train_scaled, y_train, cv=kf, scoring='r2')

mean_cross_val_mse = -cross_val_mse.mean()
std_cross_val_mse = cross_val_mse.std()
mean_cross_val_r2 = cross_val_r2.mean()
std_cross_val_r2 = cross_val_r2.std()

model_rf.fit(X_train_scaled, y_train)
y_pred_rf = model_rf.predict(X_test_scaled)

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f'Cross-Validation MSE: {mean_cross_val_mse:.4f} ± {std_cross_val_mse:.4f}')
print(f'Cross-Validation R²: {mean_cross_val_r2:.4f} ± {std_cross_val_r2:.4f}')
print(f'Test MSE: {mse_rf:.4f}')
print(f'Test RMSE: {rmse_rf:.4f}')
print(f'Test R²: {r2_rf:.4f}')

Cross-Validation MSE: 815.3489 ± 449.8001
Cross-Validation R²: -0.1372 ± 0.6212
Test MSE: 524.0398
Test RMSE: 22.8919
Test R²: -0.2243


In [161]:
import matplotlib.pyplot as plt

importances = model_rf.feature_importances_
features = X.columns

indices = np.argsort(importances)[::-1]

# plt.figure(figsize=(10, 6))
# plt.title("RF Feature Importances")
# plt.barh(range(X.shape[1]), importances[indices], align="center")
# plt.yticks(range(X.shape[1]), features[indices], rotation=0)
# plt.ylim([-1, X.shape[1]])
# plt.tight_layout()

In [154]:
import matplotlib.pyplot as plt

# plt.scatter(y_test, y_pred_rf, color='green', alpha=0.7)
# plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='k')
# plt.title('cant fit regression with only 47 dataset entry')

In [114]:
X.columns[0]

'Star Rating'